# Notebook 24: One-Way Analysis of Variance (ANOVA) Solutions
***

We'll need Numpy, Matplotlib, Pandas, and scipy.stats for this notebook, so let's load them. 

In [3]:
import numpy as np 
from scipy import stats
import statsmodels.api as sm 
import pandas as pd
import matplotlib.pylab as plt 
%matplotlib inline

### Exercise 1 - Diet, Exercise, and Weight Loss 
*** 

A randomized control study was performed with $9$ subjects to investigate the effect of exercise and diet on weight loss.  All $9$ subjects of the study exercised on a daily basis, one third of the subjects ate their regular diet, one third of subjects ate based on Diet $A$, and one third of subjects ate based on Diet $B$.  The observed weight loss after one week is summarized in the following data. 

In [14]:
dfD = pd.DataFrame({"Control": [3, 2, 1], "Diet A": [5, 3, 4], "Diet B": [5, 6, 7] })
dfD.head()

,Control,Diet A,Diet B
0,3,5,5
1,2,3,6
2,1,4,7


**Part A**: We're interested in determining whether the mean weight-loss of all three groups are the same, or if some groups have better results.  We've done this example by hand in class.  In this exercise you'll use [scipy.stats.f_oneway](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html) function to verify our results. Check out the docs, then use the function to find the appropriate $F$-statistic and corresponding p-value for the ANOVA test. Does the test indicate, at the $\alpha = 0.05$ significance level, that the mean weight-loss across all groups is the same?  

In [15]:
# stats.f_oneway returns 2 parameters from the groups of data: F-score and p-value of F.
F, pval = stats.f_oneway(dfD["Control"], dfD["Diet A"], dfD["Diet B"])

# Print out the F-score and the corresponding p-value.
print("F = {:.5f}".format(F))
print("pval = {:.5f}".format(pval))

# Find the critical F. 
# Recall the df of numerator is k-1=3-1=2 and df of denominator is N-k=9-3=6.
F_cr = stats.f.ppf(.95,2,6)

# Print the critical F-value.
print("The critical F value is {:.5}".format(F_cr))

F = 12.00000
pval = 0.00800
The critical F value is 5.1433


Since $0.008 < \alpha = 0.05$ we conclude that $\color{red}{\text{at least one of the group means is different}}$ from the others.

Also, the F-score is greater than the critical F. i.e.  $12>5.14$, so again, we reject the null and say, $\color{red}{\text{at least one of the group means is different}}$ from the others.

Note: Remember the F-score and p-value: 12.0 and .008

**Part B**: In class, we claimed that an ANOVA $F$-test is equivalent to linear regression where the features are binary categorical variables associated with group membership. In this exercise we'll verify this.

The following code re-factors the DataFrame to create binary categorical variables corresponding to Diet $A$ and Diet $B$.  Look at the resulting DataFrame, and explain how the response and features are encoded. 

In [16]:
# Create columns of the same data in order to analyze using MLR instead of ANOVA.

y = dfD.values   # .values changes data format from pandas to numpy
                 # So we have [3 5 5]
                 #            [2 3 6]
                 #            [1 4 7]
y = y.T          # .T transposes the data. Rows become columns.
                 # So, now we have [3 2 1]
                 #                 [5 3 4]
                 #                 [5 6 7]
y = y.flatten()  # .flatten produces one continuous array from the rows.
                 # So y is now the numpy array [3 2 1 5 3 4 5 6 7]
                 # Or, [Control, Diet A, Diet B]

# The same work could have been done in a single line of code as follows:
#y = dfD.values.T.flatten()


dct = {"loss": y}  # creates a single element dictionary. 
                   # One key, one value (the 'y' array).
                   # dct is now {'loss': array([3, 2, 1, 5, 3, 4, 5, 6, 7])}

# Create an empty array called 'counts'
counts = []

# This for-loop fills 'counts' with the lengths of the original data frames columns.
# The column entitled 'control' contained 3 pieces of data.
# The column entitled 'Diet A' contained 3 pieces of data.
# The column entitled 'Diet B' contained 3 pieces of data.
for column in dfD.columns:    # .columns cycles thru the headers of the dataframe
    count = len(dfD[column])  # The length of each column is stored in array 'count'
    counts.append(count)      # counts is now [3 3 3]

# Create an empty array called 'ccf'
ccf = []                            # cumulative count function

# 'num_cols' contains the number of columns of the original dataframe, i.e. 3.
num_cols = len(dfD.columns)         # .columns returns header names of dataframe

# This for-loop fills 'ccf' with cumulative values from 'counts=[3,3,3]'.
for ii in range(1,num_cols+1):      # loop runs for 1, 2, 3
    val = int(np.sum(counts[:ii]))  # 'counts' is [3, 3, 3]. iterations: 3, 3+3, 3+3+3
    ccf.append(val)                 # ccf goes from [3] to [3, 6] to [3, 6, 9]

# This for-loop runs 2 times, once for ii=1 and once for ii=2.
for ii in range(1,num_cols):    # num_cols holds the value 3 since there are 3 headers
    x = np.zeros(len(y))        # 'y' holds the 9 weight loss values. 'x' is 9 zero's.
    x[ccf[ii-1]:ccf[ii]] = 1    # first iteration from 3 to 6. Second iteration, 6 to 9.
                                   # First iteration, x[3:6] filled with '1'.
                                       # [0 0 0 1 1 1 0 0 0]
                                   # Second iteration, x[6:9] filled with '1'.
                                       # [0 0 0 0 0 0 1 1 1]
    dct[dfD.columns[ii]] = x    # recall dct is a single element (key:value) dictionary.
                                # creating new key for dictionary.
                                # New key has header from dataframe dfD.
                                # The new key was called with columns[ii]         

print(dct)                      # get a visual of the dictionary created.
dfR = pd.DataFrame(dct)         # creating a new df with the dictionary input
dfR.head(10)

{'loss': array([3, 2, 1, 5, 3, 4, 5, 6, 7], dtype=int64), 'Diet A': array([0., 0., 0., 1., 1., 1., 0., 0., 0.]), 'Diet B': array([0., 0., 0., 0., 0., 0., 1., 1., 1.])}


,loss,Diet A,Diet B
0,3,0.0,0.0
1,2,0.0,0.0
2,1,0.0,0.0
3,5,1.0,0.0
4,3,1.0,0.0
5,4,1.0,0.0
6,5,0.0,1.0
7,6,0.0,1.0
8,7,0.0,1.0


We choose the Control column as the control group (duh).  The binary features $X_1$ and $X_2$ correspond to the Diet $A$ and Diet $B$ groups, respectively.  The first three responses correspond to the control group, so both $X_1$ and $X_2$ are zero.  The next three responses correspond to Diet $A$ so $X_1 = 1$ and $X_2 = 0$.  Finally, the last three responses correspond to Diet $B$ so $X_1 = 0$ and $X_2 = 1$. 

**Part C**: Use statsmodels to $\color{red}{\text{perform a multiple linear regression on the data}}$ created in **Part B**.  Look at the model summary and compare the computed $F$-test and model coefficients to the results above.  

In [17]:
y = dfR.loc[:,"loss"]                 # The dependent variable
X = dfR.loc[:,["Diet A", "Diet B"]]   # The predictors
X = sm.add_constant(X)                # column of constants added for sm.OLS
#print(X)
model = sm.OLS(y, X).fit()
model.summary()

C:\Users\julia\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\julia\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   loss   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     12.00
Date:                Mon, 05 Dec 2022   Prob (F-statistic):            0.00800
Time:                        13:30:13   Log-Likelihood:                -10.946
No. Observations:                   9   AIC:                             27.89
Df Residuals:                       6   BIC:                             28.48
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0000      0.577      3.464      0.013       0.587       3.413
Diet A         2.0000      0.816      2.449      0.050       0.002       3.998
Diet B         4.0000      0.816      4.899      0.003       2.002       5.998
==============================================================================
Omnibus:                        2.380   Durbin-Watson:                   2.333
Prob(Omnibus):                  0.304   Jarque-Bera (JB):                0.844
Skew:                           0.000   Prob(JB):                        0.656
Kurtosis:                       1.500   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The computed $F$-statistic and associated p-value for the MLR model are identical to those from the ANOVA model. 

### Exercise 2 - Who's the Better Archer? 
*** 

Three friendly archery enthusiasts are arguing over which one is the superior archer.  Having taken Intro to Data Science, they decide to settle the bet by having a short competition and then performing a statistical analysis on the results.  In the competition, each archer takes 6 shots and records their score based on distance from the target (farther/higher is better).  The results are as follows: 

In [18]:
dfA = pd.DataFrame({"Suzie": np.array([5,4,4,3,9,4]),"Jack": np.array([4,8,7,5,1,5]),"Ruth": np.array([9,8,8,10,5,10])})
dfA.head(10)


,Suzie,Jack,Ruth
0,5,4,9
1,4,8,8
2,4,7,8
3,3,5,10
4,9,1,5
5,4,5,10


**Part A**: Use stats.f_oneway to perform an $F$-test to determine if the mean scores of the three archers are different at the $\alpha = 0.05$ significance level.  

In [19]:
# Collect the F-score and the p-value from .f_oneway.
F, pval = stats.f_oneway(dfA["Jack"], dfA["Ruth"], dfA["Suzie"])

# Just for fun, find the critical-F value, k-1 and N-k for df of num and denom.
F_crt = stats.f.ppf(.95,3-1,18-3)
print("The critical F is {:.5}".format(F_crt))

# Print the F-score and the p-value.
print("F, pval = {:.5f}, {:.5f}".format(F, pval))

The critical F is 3.6823
F, pval = 5.00000, 0.02168


Since the p-value of the ANOVA test is $0.02168 < \alpha = 0.05$ we reject the null hypothesis and conclude that there is at least one mean in the three groups that is different from the others.

Alternatively, we also reject the null since the F-score = 5 > Critical-F = 3.68. We again note that at least one of the scores is different from the others.

**Part B**: Use numpy to compute the $F$-statistic and associated p-value directly.  Verify that you get the same results as produced by stats.f_oneway.

$SSW=\sum_{k=1}^{q}\sum_{j=1}^s(x_j-\bar{x}_k)^2$

$SSB=n\cdot\sum_{k=1}^g(\bar{x}_k-\bar{\bar{x}})^2$

$MSW = \frac{SSW}{N-g}$

$MSB = \frac{SSB}{g-1}$

$F=\frac{MSB}{MSW}$

$\Large{F_{Critical} = F_{\alpha,\phantom{x} df_B,\phantom{x} df_W}}$



In [20]:
# Get total number of points, which we will need in our calculations as seen above.
# Get this number by changing the df to numpy (array), 'flatten' it, and find its length.
N = len(dfA.values.flatten())   # pandas to numpy, continuous array
  # This means N=18. All 18 scores.

# Get total number of groups, again needed in the above calculations.
I = len(dfA.columns)   # 'columns' returns the headers of the dataframe
  # This means I=3. Suzie, Jack, Ruth
    
# Compute the grand_mean using .mean on all the 'values'.
grand_mean = np.mean(dfA.values) 

# Compute the between-group sum of squares 
SSB = np.sum([dfA[col].count()*(dfA[col].mean()-grand_mean)**2 for col in dfA.columns])

# Compute the between_group degrees of freedom 
SSB_df = I-1 

# Compute the within-group sum of squares 
SSW = np.sum([np.sum((dfA[col] - dfA[col].mean())**2) for col in dfA.columns])

# Compute the within_group degrees of freedom 
SSW_df = N-I 

# Compute the test statistic 
F = (SSB/SSB_df)/(SSW/SSW_df) 

# Compute the associated p-value 
pval = 1 - stats.f.cdf(F, SSB_df, SSW_df) 

# Print the results.
print("SSB, SSB_df = {:.3f}, {}".format(SSB, SSB_df))
print("SSW, SSW_df = {:.3f}, {}".format(SSW, SSW_df))
print("F, pval = {:.5f}, {:.5f}".format(F, pval))

SSB, SSB_df = 46.778, 2
SSW, SSW_df = 70.167, 15
F, pval = 5.00000, 0.02168


The computed $F$-statistic and associated $p$-value are identical to those computed by stats.f_oneway. 

These (F and pval) are the same as given above using stats.f_oneway

**Part C**: So, we see again that we should reject the null. This means that at least one of the archers has a score that is considered significantly different than their opponents. But how should they be ranked?

Run the code below to use Tukey's Honest Significant Difference (HSD) test to determine which archers are statistically different using the [MultiComparison](http://www.statsmodels.org/dev/generated/statsmodels.sandbox.stats.multicomp.MultiComparison.html) module. Interpret the results. 

In [25]:
from statsmodels.stats.multicomp import MultiComparison

In [26]:
data = dfA.values.T.flatten()  # Pandas to NumPy, Transpose, and 'flatten'
  # data = [5 4 4 3 9 4 4 8 7 5 1 5 9 8 8 10 5 10]

# Create an empty array to hold the names of the archers.
labels = []

# This for-loop puts the archers name in once for each of their scores.
# .count() is a built-in function that returns the number of times an object
#  appears in a list. 
for col in dfA.columns:
    labels = labels + [col]*dfA[col].count()
  # labels = ['Suzie', 'Suzie', ...,'Jack', 'Jack', ...'Ruth', 'Ruth',...]
  # Each 'name' 6 times.
    
mc = MultiComparison(data, labels)
result = mc.tukeyhsd()

print(result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
  Jack   Ruth   3.3333 0.0435  0.0911  6.5755   True
  Jack  Suzie  -0.1667    0.9 -3.4089  3.0755  False
  Ruth  Suzie     -3.5 0.0337 -6.7422 -0.2578   True
----------------------------------------------------


Since there are three groups (corresponding to the three archers) we make $3$ pairwise comparisons.  The reject column tells us whether or not the null hypothesis that the two means are equal is rejected or not.  We see that there is sufficient statistical evidence to believe that Jack's and Ruth's means are different, Jack's and Suzie's means are **not** different, and Ruth's and Suzie's means are different. 

From this we can conclude that Jack's and Suzie's means are the same while Ruth's is different from the others.  Inspecting the sample means we observe that 

In [27]:
print("Suzie mean: {:.3f}".format(dfA["Suzie"].mean()))
print("Jack mean: {:.3f}".format(dfA["Jack"].mean()))
print("Ruth mean: {:.3f}".format(dfA["Ruth"].mean()))

Suzie mean: 4.833
Jack mean: 5.000
Ruth mean: 8.333


Since Ruth's sample mean is higher than the others we conclude that $\mu_{Ruth} > \mu_{Suzie} = \mu_{Jack}$.

## This is the example on schools from our notes on ANOVA.

In [24]:
dfSchools = pd.read_csv("schools.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'schools.csv'

In [ ]:
dfSchools.head(11)

In [ ]:
# Collect the F-score and the p-value from .f_oneway.
F, pval = stats.f_oneway(dfSchools["A"], dfSchools["B"], dfSchools["C"])

# Just for fun, find the critical-F value, k-1 and N-k for df of num and denom.
F_crtl = stats.f.ppf(.95,3-1,30-3)
print("The critical F is {:.5}".format(F_crtl))

# Print the F-score and the p-value.
print("F, pval = {:.5f}, {:.5f}".format(F, pval))

In [ ]:
data2 = dfSchools.values.T.flatten()  # Pandas to NumPy, Transpose, and 'flatten'

schoolNames = []

for col in dfSchools.columns:
    schoolNames = schoolNames + [col]*dfSchools[col].count()
    
mcSchool = MultiComparison(data2, schoolNames)
result_S = mcSchool.tukeyhsd()

print(result_S)

Can you decipher the comparisons?

In [ ]:
print("School A mean: {:.3f}".format(dfSchools["A"].mean()))
print("School B mean: {:.3f}".format(dfSchools["B"].mean()))
print("School C mean: {:.3f}".format(dfSchools["C"].mean()))

School A and School B are not that different.

School A and School C are not that different.

School B and School C are statistically significantly different.

The order, from low to high, is similar to:  $C$ then $A$ then $B$.

In [ ]:
stats.f.ppf(.95,2,6)